In [1]:
from util import *
df = load_bout()
users = getSortedUser(df)
n_user = len(users)

In [17]:
df_ = df.query("weekday < 5")
watch = df_.query("btype!= 'p'").groupby(["uid","date", "hour"]).agg(wstep = ("step","sum"), wcnt = ("step","count"))
total = df_.groupby(["uid","date", "hour"]).agg(step = ("step","sum"), cnt = ("step","count"))
tmp = pd.concat([watch,total],axis = 1).fillna(0)
tmp = tmp.reset_index()
train = []
valid = []
for id in set(df_["uid"]):
    udf = tmp.query("uid == @id")
    ntrain = round(udf.shape[0] * 0.75)
    train.append(udf[:ntrain])
    valid.append(udf[ntrain:])
train = pd.concat(train)
valid = pd.concat(valid)

In [24]:
# Basic Regression
import statsmodels.api as sm

params = ['wstep']
X = sm.add_constant(train[params])
model = sm.OLS(train['step'],X)
results = model.fit()
print(results.summary())

for data in [train,valid]:
    X = sm.add_constant(data[params])
    pred = model.predict(params = results.params, exog = X)
    print(np.sqrt(np.mean((data['step'].to_numpy() - pred) **2)))



                            OLS Regression Results                            
Dep. Variable:                   step   R-squared:                       0.872
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                 1.399e+05
Date:                Thu, 27 Jan 2022   Prob (F-statistic):               0.00
Time:                        10:41:38   Log-Likelihood:            -1.4845e+05
No. Observations:               20547   AIC:                         2.969e+05
Df Residuals:                   20545   BIC:                         2.969e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         88.3073      2.606     33.882      0.0

In [23]:
# MultiLevel Regression
# Random effect 1: uid
import statsmodels.formula.api as smf

params = ['wstep']
model = smf.mixedlm("step ~ wstep", data = train, groups = train['uid'])

results = model.fit()
print(results.summary())

for data in [train,valid]:
    pred = results.predict(exog = data[params])#groups = data['uid'])
    print(np.sqrt(np.mean((data['step'].to_numpy() - pred) **2)))

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: step        
No. Observations: 20547   Method:             REML        
No. Groups:       84      Scale:              98758.3941  
Min. group size:  80      Log-Likelihood:     -147449.3826
Max. group size:  407     Converged:          Yes         
Mean group size:  244.6                                   
----------------------------------------------------------
             Coef.   Std.Err.    z    P>|z| [0.025  0.975]
----------------------------------------------------------
Intercept     87.589   12.459   7.030 0.000 63.170 112.008
wstep          0.979    0.003 385.091 0.000  0.974   0.984
Group Var  12495.789    6.407                             

332.5321616364473
365.5085351118036


In [25]:
# MultiLevel Regression
# Random effect 1: hour
import statsmodels.formula.api as smf

params = ['wstep']
model = smf.mixedlm("step ~ wstep", data = train, groups = train['hour'])

results = model.fit()
print(results.summary())

for data in [train,valid]:
    pred = results.predict(exog = data[params])#groups = data['uid'])
    print(np.sqrt(np.mean((data['step'].to_numpy() - pred) **2)))

/home/highjun/.local/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/highjun/.local/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/home/highjun/.local/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/highjun/.local/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(


          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: step        
No. Observations: 20547   Method:             REML        
No. Groups:       24      Scale:              109988.9338 
Min. group size:  181     Log-Likelihood:     -148444.2799
Max. group size:  1243    Converged:          No          
Mean group size:  856.1                                   
----------------------------------------------------------
             Coef.   Std.Err.    z    P>|z| [0.025  0.975]
----------------------------------------------------------
Intercept     84.496    9.668   8.740 0.000 65.547 103.444
wstep          0.964    0.003 369.749 0.000  0.959   0.969
Group Var   2037.139                                      

332.36750163951615
365.8116516286838


/home/highjun/.local/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/highjun/.local/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2206: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
/home/highjun/.local/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2218: ConvergenceWarning: Gradient optimization failed, |grad| = 104.719269
  warnings.warn(msg, ConvergenceWarning)
/home/highjun/.local/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
